In [1]:
import paho.mqtt.client as mqtt
import os
import pandas as pd
import joblib
import platform
from tqdm import tqdm
import json
from datetime import datetime

In [2]:
system_config = {
    "is_monitoring": False
}
connected_devices = {}
device_params = {
    "features": None,
    "state": None,
    "timestamp": None
}

In [3]:
class AnalyseSensorData():
    
    models      = None 
    names       = None
    predictions = None
    
    def __init__(self, sensor_data):
        
        self.names = list()
        self.models = dict()
        self.predictions = dict()
        self.dst = self.process_dataset(sensor_data)

        self.load_models()
        self.make_prediction()

    def get_source_directory(self):
        root = os.getcwd()
        return os.path.join(root, "models")

    def process_dataset(self, sensor_data):
        json_data = list()
        json_data.append(sensor_data)
        df = pd.DataFrame(json_data)
        X = df.to_numpy()
        return X
    
    def retrieve_name(self, file_name):
        if  platform.system() == 'Windows':
            file_name = file_name.split("\\")[-1]
        else: 
            file_name = file_name.split("/")[-1]

        period_index = file_name.rfind(".")
        return file_name[:period_index]

    def load_models(self):
        model_directory = self.get_source_directory()
        try:
            model_list = os.listdir(model_directory)
            if len(model_list) > 0:
                for file in model_list:
                    name = self.retrieve_name(file)
                    self.models[name] = joblib.load(os.path.join(model_directory, file))
                    self.names.append(name)
            else:
                raise ValueError('No saved model found in "{}" directory.'.format(source_dir))
        except Exception as e:
            print("FATAL MODEL LOAD ERROR :  ", dir(e))
            exit()
            
    def make_prediction(self):
        try:
            for name in self.names:
                self.predictions[name] = self.models[name].predict(self.dst)
        except Exception as e:
            print("FATAL PREDICTION ERROR:  ", dir(e))
            exit()

    @staticmethod
    def make_predictions(sensor_data):
        analysis = AnalyseSensorData(sensor_data)
        num_of_models = len(list(analysis.predictions.keys()))
        sum_of_predictions = sum(analysis.predictions.values())
        if num_of_models > 0:
            prediction = sum_of_predictions / num_of_models
            return 1 if prediction >= 0.5 else 0
        else:
            raise ValueError("FATAL ERROR: NO TRAINED ML MODEL FOUND!!!")
            exit()

In [4]:
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, reason_code, properties):
    print(f"Connected with result code {reason_code}")
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("Remote/FERS/IoT-Data/json")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    # Convert the string to a JSON object
    json_data = json.loads(msg.payload)

    node = json_data["Node"]
    node_timestamp = json_data["Timestamp"]
    current_timestamp = datetime.now()

    

    if not node in list(connected_devices.keys()):
        connected_devices[node] = device_params

    for key in ["Timestamp", "Node"]:
        if key in json_data.keys():
            del json_data[key]
            
    predicted = AnalyseSensorData.make_predictions(json_data)

    connected_devices[node]["features"] = json_data
    connected_devices[node]["state"] = predicted
    connected_devices[node]["timestamp"] = timestamp

    print(connected_devices)

In [ ]:
mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
mqttc.on_connect = on_connect
mqttc.on_message = on_message
mqttc.connect("broker.emqx.io", 1883, 60)
mqttc.loop_forever()

Connected with result code Success
{'123456789': {'features': {'LPG': 0.81, 'CO': 5.42, 'Smoke': 5.86, 'Temperature': 48.61, 'Humidity': 8.83}, 'state': 1, 'timestamp': '2024-11-24T06:00:23'}}
{'123456789': {'features': {'LPG': 0.81, 'CO': 5.42, 'Smoke': 5.86, 'Temperature': 48.61, 'Humidity': 8.83}, 'state': 1, 'timestamp': '2024-11-24T06:00:29'}}
{'123456789': {'features': {'LPG': 0.81, 'CO': 5.42, 'Smoke': 5.86, 'Temperature': 48.61, 'Humidity': 8.83}, 'state': 1, 'timestamp': '2024-11-24T06:00:34'}}
{'123456789': {'features': {'LPG': 0.81, 'CO': 5.42, 'Smoke': 5.86, 'Temperature': 48.61, 'Humidity': 8.83}, 'state': 1, 'timestamp': '2024-11-24T06:00:40'}}
{'123456789': {'features': {'LPG': 0.81, 'CO': 5.42, 'Smoke': 5.86, 'Temperature': 48.61, 'Humidity': 8.83}, 'state': 1, 'timestamp': '2024-11-24T06:00:45'}}
{'123456789': {'features': {'LPG': 0.81, 'CO': 5.42, 'Smoke': 5.86, 'Temperature': 48.61, 'Humidity': 8.83}, 'state': 1, 'timestamp': '2024-11-24T06:00:51'}}
{'123456789': {'f

In [4]:
from datetime import datetime, timezone

# Get the current GMT datetime
gmt_datetime = datetime.now(timezone.utc)

print("Current GMT datetime:", gmt_datetime)

# using now() to get current time
current_time = datetime.now()

# Printing value of now.
print("Time now at greenwich meridian is:", current_time)

Current GMT datetime: 2024-11-25 04:40:56.154596+00:00
Time now at greenwich meridian is: 2024-11-25 05:40:56.155593
